In [1]:
import numpy as np
import glob, os, csv
import BioSimSpace as BSS

INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
input_dir = f"/Users/guilherme/Documents/006.Research/2024_Eg5_relative_free_energies"
path_to_ligands = f"/Users/guilherme/Documents/006.Research/2024_Eg5_relative_free_energies/002.ligands/"
path_to_protein = f"/Users/guilherme/Documents/006.Research/2024_Eg5_relative_free_energies/001.protein_file/1q0b.rec.withH.pdb"

In [3]:
#BSS.Notebook.View(path_to_protein).system()

In [4]:
node = BSS.Gateway.Node("A node to create input files for molecular dynamics simulation.")
node.addInput("Ligand FF", BSS.Gateway.String( help="Force field to parameterise ligands with.", allowed=["GAFF1", "GAFF2"],default="GAFF2",),)
node.addInput("Protein FF", BSS.Gateway.String(help="Force field to parameterise the protein with.", allowed=["FF03", "FF14SB", "FF99", "FF99SB", "FF99SBILDN"], default="FF14SB",),)
node.addInput("Water Model", BSS.Gateway.String(help="Water model to use.", allowed=["SPC", "SPCE", "TIP3P", "TIP4P", "TIP5P"], default="TIP3P",),)
node.addInput("Box Edges", BSS.Gateway.String(help="Size of water box around molecular system.", allowed=["20*angstrom","25*angstrom","30*angstrom","35*angstrom","45*angstrom","5*nm","7*nm","10*nm",],default="45*angstrom",),)
node.addInput("Box Shape", BSS.Gateway.String(help="Geometric shape of water box.", allowed=["cubic", "truncatedOctahedron"], default="cubic",),)
node.addInput("Run Time", BSS.Gateway.String(help="The sampling time per lambda window.", allowed=["10*ps","100*ps","1*ns","2*ns","3*ns","4*ns","5*ns","8*ns","10*ns","12*ns","15*ns",],default="10*ns",),)

# add SOMD2 as an FEP engine option
fep_engines = [e.upper() for e in BSS.FreeEnergy.engines()]
if "SOMD2" not in fep_engines:
    fep_engines.append("SOMD2")

node.addInput("FEP Engine", BSS.Gateway.String(help="Engine to run FEP with.", allowed=fep_engines, default="GROMACS",),)
node.addInput("LambdaWindows", BSS.Gateway.String( help="The number of lambda windows for regular transformations.", allowed=["3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20",],default="15",),)
node.addInput("DiffLambdaWindows", BSS.Gateway.String(help="The number of lambda windows for difficult transformations.",allowed=["4","5","6","7","8","9","10","11","12", "13","14","15","16","17", "18","19","20",],default="20",),)
node.addInput("LOMAP Threshold", BSS.Gateway.String(help="The LOMAP score threshold to define difficult transformations.", allowed=["0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9"],default="0.4",),)
#node.showControls()

In [5]:
ligand_files = sorted(glob.glob(f"{path_to_ligands}_sdfs/*.sdf"))

In [6]:
ligands = []
ligand_names = []

for filepath in ligand_files:
    # append the molecule object to a list.
    ligands.append(BSS.IO.readMolecules(filepath)[0])
    # append the molecule name to another list so that we can use the name of each molecule in our workflow.
    ligand_names.append(filepath.split("/")[-1].replace(".sdf", ""))

transformations, lomap_scores = BSS.Align.generateNetwork(ligands, plot_network=False, names=ligand_names, work_dir="003.lomap_network/visualise_network",)

INFO:root:------------------------------
INFO:root:ID 0	000_alpha.sdf
INFO:root:ID 1	001_dc01.sdf
INFO:root:ID 2	002_dc02.sdf
INFO:root:ID 3	003_dc04.sdf
INFO:root:ID 4	004_dc05.sdf
INFO:root:ID 5	005_new01.sdf
INFO:root:ID 6	006_new02.sdf
INFO:root:ID 7	007_new03.sdf
INFO:root:ID 8	008_new04.sdf
INFO:root:------------------------------
INFO:root:Finish reading input files. 9 structures in total....skipped 0

INFO:root:
Matrix scoring in progress....

INFO:root:Parallel mode is on
INFO:root:
Generating graph in progress....
INFO:root:Trying to remove edge 6-7 with similarity 0.704688
INFO:root:Rejecting edge deletion on cycle covering
INFO:root:Trying to remove edge 1-6 with similarity 0.740818
INFO:root:Rejecting edge deletion on cycle covering
INFO:root:Trying to remove edge 3-7 with similarity 0.860708
INFO:root:Rejecting edge deletion on cycle covering
INFO:root:Trying to remove edge 1-3 with similarity 0.904837
INFO:root:Rejecting edge deletion on cycle covering
INFO:root:Trying t

In [7]:
pert_network_dict = {}
transformations_named = [(ligand_names[transf[0]], ligand_names[transf[1]]) for transf in transformations]
for score, transf in sorted(zip(lomap_scores, transformations_named)):
    pert_network_dict[transf] = score
    print(transf, score)

('dc01', 'dc05') 0.02024
('dc04', 'new04') 0.03012
('dc01', 'new04') 0.03329
('dc01', 'dc02') 0.07427
('dc02', 'dc04') 0.08086
('new01', 'new03') 0.08208
('dc01', 'new01') 0.08629
('dc02', 'dc05') 0.18268
('alpha', 'new01') 0.70469
('new02', 'new03') 0.70469
('dc01', 'new02') 0.74082
('dc04', 'new03') 0.86071
('dc01', 'dc04') 0.90484


In [8]:
prot = BSS.IO.readPDB(path_to_protein, pdb4amber=False)[0]
# depending on the picked forcefield and your protein, tleap might fail. Check work_dir to see error logs.
prot_p = BSS.Parameters.parameterise(prot, node.getInput("Protein FF")).getMolecule()
BSS.IO.saveMolecules("004.inputs/protein/protein", prot_p, ["PRM7", "RST7"])

['/Users/guilherme/Documents/006.Research/2024_Eg5_relative_free_energies/004.inputs/protein/protein.prm7',
 '/Users/guilherme/Documents/006.Research/2024_Eg5_relative_free_energies/004.inputs/protein/protein.rst7']

In [9]:
# write ligands file.
with open("003.lomap_network/ligands.dat", "w") as ligands_file:
    writer = csv.writer(ligands_file)
    for lig in ligand_names:
        writer.writerow([lig])

# write perts file. Base the lambda schedule on the file generated in the previous cell.
np.set_printoptions(formatter={"float": "{: .4f}".format})

# from protocol, derive the engine we want to use on the cluster.
engine = node.getInput("FEP Engine").upper()

with open("003.lomap_network/network.dat", "w") as network_file:
    writer = csv.writer(network_file, delimiter=" ")

    for pert, lomap_score in pert_network_dict.items():
        # based on the provided (at top of notebook) lambda allocations and LOMAP threshold, decide allocation.
        if lomap_score == None or lomap_score < float(node.getInput("LOMAP Threshold")):
            num_lambda = node.getInput("DiffLambdaWindows")
        else:
            num_lambda = node.getInput("LambdaWindows")

        # given the number of allocated lambda windows, generate an array for parsing downstream.
        lam_array_np = np.around(np.linspace(0, 1, int(num_lambda)), decimals=5)

        # make the array into a format readable by bash.
        lam_array = (
            str(lam_array_np)
            .replace("[ ", "")
            .replace("]", "")
            .replace("  ", ",")
            .replace("\n", "")
        )

        # write out both directions for this perturbation.
        writer.writerow([pert[0], pert[1], len(lam_array_np), lam_array, engine])
        writer.writerow([pert[1], pert[0], len(lam_array_np), lam_array, engine])

# create protocol.
protocol = [
    f"ligand forcefield = {node.getInput('Ligand FF')}",
    f"protein forcefield = {node.getInput('Protein FF')}",
    f"solvent = {node.getInput('Water Model')}",
    f"box edges = {node.getInput('Box Edges')}",
    f"box type = {node.getInput('Box Shape')}",
    f"protocol = default",
    f"sampling = {node.getInput('Run Time')}",
    f"engine = {node.getInput('FEP Engine').upper()}",
]

# write protocol to file.
with open("003.lomap_network/protocol.dat", "w") as protocol_file:
    writer = csv.writer(protocol_file)

    for prot_line in protocol:
        writer.writerow([prot_line])

In [10]:
main_dir = "/Users/guilherme/Documents/006.Research/2024_Eg5_relative_free_energies/003.lomap_network"
protein_file = "/Users/guilherme/Documents/006.Research/2024_Eg5_relative_free_energies/004.inputs/protein/protein"
for ligand in ligand_files:
    lig_name = ligand.split("/")[-1][:-4]
    ### Load matching input with BSS.read.IO.
    lig = BSS.IO.readMolecules(f"{ligand}")[0]
    #################
    ### parameterise ligand by deriving requested FF from protocol.dat.
    stream = open(f"{main_dir}/protocol.dat", "r")
    lines = stream.readlines()
    ff_query = lines[0].rstrip()

    # do tests on protocol, parameterise with correct FF.
    if not "ligand" in ff_query:
        raise NameError(
            "Please supply a ligand force field on the first line of protocol.dat. The line should look like (e.g.):\n"
            + "ligand forcefield = GAFF2"
        )
    else:
        if "GAFF1" in ff_query:
            print(f"Parameterising {lig_name} using GAFF1 force field.")
            lig_p = BSS.Parameters.gaff(lig).getMolecule()

        elif "GAFF2" in ff_query:
            print(f"Parameterising {lig_name} using GAFF2 force field.")
            lig_p = BSS.Parameters.gaff2(lig).getMolecule()
        else:
            raise NameError(
                f"Force field not supported: {ff_query}. Please use either of [GAFF1, GAFF2, OpenForceField], or"
                + " edit this script to use other force fields available in BSS.Parameters.forceFields()."
            )
    BSS.IO.saveMolecules(f"004.inputs/ligands/{lig_name}/{lig_name}", lig_p, ["PRM7", "RST7"])
    # at this point BSS should have thrown an error if parameterisation failed, so no checks needed.
    ### make a copy of ligand, solvate original ligand object.
    lig_p_copy = lig_p.copy()

    # need to derive some settings from protocol.dat again.
    stream = open(f"{main_dir}/protocol.dat", "r")
    lines = stream.readlines()

    # get the solvent force field.
    solvent_query = lines[2].rstrip().replace(" ", "").split("=")[-1]

    # get the box size settings.
    boxsize_query = lines[3].rstrip().replace(" ", "").split("=")[-1]
    box_axis_length = boxsize_query.split("*")[0]
    box_axis_unit = boxsize_query.split("*")[1]
    if box_axis_unit.lower() == "nm" or box_axis_unit.lower() == "nanometer":
        box_axis_unit = BSS.Units.Length.nanometer

    elif box_axis_unit.lower() == "a" or box_axis_unit.lower() == "angstrom":
        box_axis_unit = BSS.Units.Length.angstrom
    else:
        raise NameError(
            "Input unit not recognised. Please use any of ['spc', 'spce', 'tip3p', 'tip4p', 'tip5p'] in "
            + "the fourth line of protocol.dat in the shape of (e.g.):\nbox edges = 10*angstrom"
        )


    """
figure out the ligand dimensions, add the specified water box together with the largest ligand axis.
This is a workaround to account for adding ions in some cases. Based on:
https://github.com/michellab/BioSimSpace/issues/111
"""
    box_min, box_max = lig_p.getAxisAlignedBoundingBox()
    box_size = [y - x for x, y in zip(box_min, box_max)]
    box_sizes = [x + int(box_axis_length) * box_axis_unit for x in box_size]


    # do the same for ligand +protein system.
    protein = BSS.IO.readMolecules([f"{protein_file}.rst7", f"{protein_file}.prm7"])[0]
    system = lig_p + protein

    # box size
    boxsize_query = lines[3].rstrip().replace(" ", "").split("=")[-1]
    box_axis_length = boxsize_query.split("*")[0]
    box_axis_unit = boxsize_query.split("*")[1]
    if box_axis_unit.lower() == "nm" or box_axis_unit.lower() == "nanometer":
        box_axis_unit = BSS.Units.Length.nanometer
    elif box_axis_unit.lower() == "a" or box_axis_unit.lower() == "angstrom":
        box_axis_unit = BSS.Units.Length.angstrom
    else:
        raise NameError(
            "Input unit not recognised. Please use any of ['spc', 'spce', 'tip3p', 'tip4p', 'tip5p'] in "
            + "the fourth line of protocol.dat in the shape of (e.g.):\nbox edges = 10*angstrom"
        )

    # get the box type settings.
    boxtype_query = lines[4].rstrip().replace(" ", "").split("=")[-1]

    boxtype_dict = {
        "cubic": BSS.Box.cubic,
        "truncatedoctahedron": BSS.Box.truncatedOctahedron,
        "octahedral": BSS.Box.truncatedOctahedron,
    }

    if boxtype_query not in boxtype_dict:
        raise NameError(
            "Input box type not recognised. Please use any of ['orthorhombic', 'octahedral', 'triclinic']"
            + "in the fifth line of protocol.dat in the shape of (e.g.):\nbox type = orthorhombic"
        )


    boxtype_func = boxtype_dict[boxtype_query]
    print(f"Solvating for {lig_name}...")

    box_min, box_max = lig_p.getAxisAlignedBoundingBox()
    box_size = [y - x for x, y in zip(box_min, box_max)]
    box_sizes = [x + int(box_axis_length) * box_axis_unit for x in box_size]

    box, angles = boxtype_func(max(box_sizes))
    lig_p_solvated = BSS.Solvent.solvate(
        solvent_query, molecule=lig_p, box=box, angles=angles, ion_conc=0.15
    )
    BSS.IO.saveMolecules(f"004.inputs/ligands_solvated/{lig_name}/{lig_name}", lig_p_solvated, ["PRM7", "RST7"])

    box_min, box_max = system.getAxisAlignedBoundingBox()
    box_size = [y - x for x, y in zip(box_min, box_max)]
    box_sizes = [x + int(box_axis_length) * box_axis_unit for x in box_size]

    box, angles = boxtype_func(max(box_sizes))
    system_solvated = BSS.Solvent.solvate(
        solvent_query, molecule=system, box=box, angles=angles, ion_conc=0.15
    )
    BSS.IO.saveMolecules(f"004.inputs/systems/{lig_name}/{lig_name}", system_solvated, ["PRM7", "RST7"])

Parameterising alpha using GAFF2 force field.
Solvating for alpha...
Parameterising dc01 using GAFF2 force field.
Solvating for dc01...
Parameterising dc02 using GAFF2 force field.
Solvating for dc02...
Parameterising dc04 using GAFF2 force field.
Solvating for dc04...
Parameterising dc05 using GAFF2 force field.
Solvating for dc05...
Parameterising new01 using GAFF2 force field.
Solvating for new01...
Parameterising new02 using GAFF2 force field.
Solvating for new02...
Parameterising new03 using GAFF2 force field.
Solvating for new03...
Parameterising new04 using GAFF2 force field.
Solvating for new04...


In [11]:
main_dir = os.getcwd()

### first, figure out which engine and what runtime the user has specified in protocol.
stream = open(f"{main_dir}/003.lomap_network/protocol.dat", "r")
lines = stream.readlines()

### get the requested engine.
engine_query = lines[7].rstrip().replace(" ", "").split("=")[-1].upper()
if engine_query not in ["GROMACS"]:
    raise NameError(
        """Input MD engine not recognised.
        Please use 'GROMACS' on the eighth line 
        of protocol.dat in the shape of (e.g.):
        engine = GROMACS""")

### get the requested runtime.
runtime_query = lines[6].rstrip().replace(" ", "").split("=")[-1].split("*")[0]
try:
    runtime_query = int(runtime_query)
except ValueError:
    raise NameError("Input runtime value not supported. Please use an integer on the seventh line of protocol.dat in the shape of (e.g.):\nsampling = 2*ns")

# make sure user has set ns or ps.
runtime_unit_query = lines[6].rstrip().replace(" ", "").split("=")[-1].split("*")[1]
if runtime_unit_query not in ["ns", "ps"]:
    raise NameError("Input runtime unit not supported. Please use 'ns' or 'ps' on the seventh line of protocol.dat in the shape of (e.g.):\nsampling = 2*ns")

if runtime_unit_query == "ns":
    runtime_unit = BSS.Units.Time.nanosecond
elif runtime_unit_query == "ps":
    runtime_unit = BSS.Units.Time.picosecond

####### Prepare simulation files
with open(f"{main_dir}/003.lomap_network/network.dat") as lambdas_file:
    reader = csv.reader(lambdas_file, delimiter=" ")
    for row in reader:
        lig_0, lig_1 = row[0], row[1]
        num_lambda = int(row[2])
        # Load equilibrated free inputs for both ligands. Complain if input not found. These systems already contain equil. waters.
        print(f"Loading ligands {lig_0} and {lig_1}.")
        ligs_path = f"{main_dir}/004.inputs/ligands_solvated/"
        ligand_1_sys = BSS.IO.readMolecules([f"{ligs_path}{lig_0}/{lig_0}.rst7", f"{ligs_path}{lig_0}/{lig_0}.prm7",])
        ligand_2_sys = BSS.IO.readMolecules([f"{ligs_path}{lig_1}/{lig_1}.rst7", f"{ligs_path}{lig_1}/{lig_1}.prm7",])
        
        # Extract ligands.
        ligand_1 = ligand_1_sys.getMolecule(0)
        ligand_2 = ligand_2_sys.getMolecule(0)
        
        # Align ligand2 on ligand1
        print("Mapping and aligning..")
        print(ligand_1, ligand_2)
        mapping = BSS.Align.matchAtoms(ligand_1, ligand_2, complete_rings_only=True)
        inv_mapping = {v: k for k, v in mapping.items()}
        ligand_2_a = BSS.Align.rmsdAlign(ligand_2, ligand_1, inv_mapping)
        
        # Generate merged molecule.
        print("Merging..")
        merged_ligs = BSS.Align.merge(ligand_1, ligand_2_a, mapping)
        
        #### Get equilibrated waters and waterbox information for both bound and free. Get all information from lambda==0
        # Following is work around because setBox() doesn't validate correctly boxes with lengths and angles
        
        ligand_1_sys.removeMolecules(ligand_1)
        ligand_1_sys.addMolecules(merged_ligs)
        system_free = ligand_1_sys
        
        
        ################ now repeat above steps, but for the protein + ligand systems.
        # Load equilibrated bound inputs for both ligands. Complain if input not found
        print(f"Loading bound ligands {lig_0} and {lig_1}.")
        ligs_path = f"{main_dir}/004.inputs/systems/"
        system_1 = BSS.IO.readMolecules([f"{ligs_path}{lig_0}/{lig_0}.rst7", f"{ligs_path}{lig_0}/{lig_0}.prm7",])
        system_2 = BSS.IO.readMolecules([f"{ligs_path}{lig_1}/{lig_1}.rst7", f"{ligs_path}{lig_1}/{lig_1}.prm7",])
        
        # Extract ligands and protein. Do this based on nAtoms and nResidues, as sometimes
        # the order of molecules is switched, so we can't use index alone.
        # bugfix in BSS makes the below redundant but keeping this in to be 100% sure we're getting the correct structures.
        system_ligand_1 = None
        protein = None
        n_residues = [mol.nResidues() for mol in system_1]
        n_atoms = [mol.nAtoms() for mol in system_1]
        for i, (n_resi, n_at) in enumerate(zip(n_residues[:20], n_atoms[:20])):
            if n_resi == 1 and n_at > 5:
                system_ligand_1 = system_1.getMolecule(i)
            elif n_resi > 1:
                protein = system_1.getMolecule(i)
            else:
                pass
        
        # loop over molecules in system to extract the ligand
        system_ligand_2 = None
        
        n_residues = [mol.nResidues() for mol in system_2]
        n_atoms = [mol.nAtoms() for mol in system_2]
        for i, (n_resi, n_at) in enumerate(zip(n_residues, n_atoms)):
            # grab the system's ligand and the protein. ignore the waters.
            if n_resi == 1 and n_at > 5:
                system_ligand_2 = system_2.getMolecule(i)
            else:
                pass
        
        # extract ions.
        # ions_bound = system_2.search("not mols with atomidx 2")
        
        if system_ligand_1 and system_ligand_2 and protein:
            print("Using molecules ligand_1, ligand_2, protein:")
            print(system_ligand_1, system_ligand_2, protein)
        else:
            raise _Exceptions.AlignmentError("Could not extract ligands or protein from input systems. Check that your ligands/proteins are properly prepared by BSSligprep.sh!")
        
        # Align ligand2 on ligand1
        print("Mapping..")
        mapping = BSS.Align.matchAtoms(system_ligand_1, system_ligand_2, complete_rings_only=True)
        inv_mapping = {v: k for k, v in mapping.items()}
        
        print("Aligning..")
        system_ligand_2_a = BSS.Align.rmsdAlign(system_ligand_2, system_ligand_1, inv_mapping)
        
        # Generate merged molecule.
        print("Merging..")
        system_merged_ligs = BSS.Align.merge(system_ligand_1, system_ligand_2_a, mapping)
        
        system_1.removeMolecules(system_ligand_1)
        system_1.addMolecules(system_merged_ligs)
        system_bound = system_1

        min_protocol = BSS.Protocol.FreeEnergyMinimisation(num_lam=num_lambda)
        
        # define the free energy protocol with all this information. User could customise settings further here, see docs.
        #freenrg_protocol = BSS.Protocol.FreeEnergy(
        #    num_lam=num_lambda, runtime=runtime_query * runtime_unit
        #)

        # # for GROMACS, also need per lambda min + eq
        # if engine_query == "GROMACS":
        #     min_protocol = BSS.Protocol.FreeEnergyMinimisation(num_lam=num_lambda)
        #     #nvt_protocol = BSS.Protocol.FreeEnergyEquilibration(
        #     #    num_lam=num_lambda, pressure=None, temperature= 298.15 * BSS.Units.Pressure.kelvin
        #     #)
        #     #npt_protocol = BSS.Protocol.FreeEnergyEquilibration(
        #     #    num_lam=num_lambda, pressure=1 * BSS.Units.Pressure.atm, temperature= 298.15 * BSS.Units.Pressure.kelvin
            #)
        
        ############# Set up the directory environment.
        workdir = f"{main_dir}/005.RBFE/{lig_0}~{lig_1}/"
        print(f"Setting up {engine_query} directory environment in {workdir}.")
        
        print("Bound..")
        workdir = f"{main_dir}/005.RBFE/{lig_0}~{lig_1}"
        BSS.FreeEnergy.Relative(
            system_bound,
            min_protocol,
            engine=f"{engine_query}",
            work_dir=workdir + "/bound/",
        )
    
        print("Free..")
        workdir = f"{main_dir}/005.RBFE/{lig_0}~{lig_1}"
        BSS.FreeEnergy.Relative(
            system_free,
            min_protocol,
            engine=f"{engine_query}",
            work_dir=workdir + "/free/",
        )
    
    

Loading ligands dc01 and dc05.
Mapping and aligning..
<BioSimSpace.Molecule: number=803155, nAtoms=47, nResidues=1> <BioSimSpace.Molecule: number=808887, nAtoms=55, nResidues=1>
Merging..
Loading bound ligands dc01 and dc05.
Using molecules ligand_1, ligand_2, protein:
<BioSimSpace.Molecule: number=814623, nAtoms=47, nResidues=1> <BioSimSpace.Molecule: number=853371, nAtoms=55, nResidues=1> <BioSimSpace.Molecule: number=822502, nAtoms=5235, nResidues=331>
Mapping..
Aligning..
Merging..
Setting up GROMACS directory environment in /Users/guilherme/Documents/006.Research/2024_Eg5_relative_free_energies/005.RBFE/dc01~dc05/.
Bound..
Free..
Loading ligands dc05 and dc01.
Mapping and aligning..
<BioSimSpace.Molecule: number=892071, nAtoms=55, nResidues=1> <BioSimSpace.Molecule: number=897788, nAtoms=47, nResidues=1>
Merging..
Loading bound ligands dc05 and dc01.
Using molecules ligand_1, ligand_2, protein:
<BioSimSpace.Molecule: number=903543, nAtoms=55, nResidues=1> <BioSimSpace.Molecule: nu